In [1]:
!pip install dash
!pip install plotly

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 228.0/228.0 kB 13.4 MB/s eta 0:00:00
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 3.1.3
    Uninstalling Werkzeug-3.1.3:
      Successfully uninstalled Werkzeug-3.1.3
  Attempting uninstall: Flask
    Found existing installation: Flask 3.1.1
    Uninstalling Flask-3.1.1:
      Successfully uninstalled Flask-3.1.1


In [3]:
import pandas as pd
import plotly.express as px
from dash import Dash, dcc, html, Input, Output

# Gunakan nama fiile sesuai yang diunggah (hapus karakter aneh)
file_path = "https://docs.google.com/spreadsheets/d/e/2PACX-1vRrWlRl2tdI1pAr_otY-9tOTWS7LB2SCF0ay7pkAncCqxdElbEnlepECM4LlqQtheIAEm1WIULPtyal/pub?output=xlsx"
df = pd.read_excel(file_path)

# Ubah format dari lebar ke panjang
df_melted = df.melt(id_vars=["Kabupaten"], var_name="Jenis_Tahun", value_name="Jumlah")
df_melted[['Jenis_Kejahatan', 'Tahun']] = df_melted['Jenis_Tahun'].str.extract(r'Kejahatan Terhadap (.*?) (\d{4})')
df_melted.drop(columns='Jenis_Tahun', inplace=True)

# Bersihkan dan konversi tipe
df_melted['Jenis_Kejahatan'] = df_melted['Jenis_Kejahatan'].str.strip()
df_melted['Tahun'] = pd.to_numeric(df_melted['Tahun'], errors='coerce')
df_melted['Jumlah'] = pd.to_numeric(df_melted['Jumlah'], errors='coerce')

# Tambahkan koordinat kabupaten di Lampung
koordinat = {
    'Bandar Lampung': (-5.3971, 105.2668),
    'Lampung Selatan': (-5.5621, 105.6203),
    'Lampung Tengah': (-4.8942, 105.0946),
    'Lampung Utara': (-4.8256, 104.7607),
    'Lampung Timur': (-5.1139, 105.6132),
    'Pringsewu': (-5.3586, 104.9762),
    'Pesawaran': (-5.5004, 105.0796),
    'Tanggamus': (-5.4945, 104.5648),
    'Tulang Bawang': (-4.2034, 105.6431),
    'Tulang Bawang Barat': (-4.4094, 104.8015),
    'Way Kanan': (-4.5006, 104.3212),
    'Mesuji': (-3.7279, 105.3317),
    'Metro': (-5.1135, 105.3072),
    'Pesisir Barat': (-5.0540, 103.9461),
}

df_koordinat = pd.DataFrame([
    {"Kabupaten": k, "Latitude": v[0], "Longitude": v[1]} for k, v in koordinat.items()
])

# Gabungkan dengan data utama
df_melted = df_melted.merge(df_koordinat, on="Kabupaten", how="left")

# Inisialisasi aplikasi Dash
app = Dash(__name__)
app.title = "Dashboard Kejahatan Lampung"

# Layout aplikasi
app.layout = html.Div([
    html.H2("Dashboard Kejahatan di Lampung (2020–2023)", style={'textAlign': 'center'}),

    html.Div([
        html.Label("Pilih Tahun:"),
        dcc.Dropdown(
            id='dropdown-tahun',
            options=[{'label': str(t), 'value': t} for t in sorted(df_melted['Tahun'].dropna().unique())],
            value=sorted(df_melted['Tahun'].dropna().unique())[0]
        ),
    ], style={'width': '48%', 'display': 'inline-block'}),

    html.Div([
        html.Label("Pilih Jenis Kejahatan:"),
        dcc.Dropdown(
            id='dropdown-kejahatan',
            options=[{'label': k, 'value': k} for k in sorted(df_melted['Jenis_Kejahatan'].dropna().unique())],
            value=sorted(df_melted['Jenis_Kejahatan'].dropna().unique())[0]
        ),
    ], style={'width': '48%', 'display': 'inline-block', 'float': 'right'}),

    html.Br(), html.Br(),

    dcc.Graph(id='grafik-kejahatan'),
    dcc.Graph(id='peta-kejahatan')
])

# Callback interaktif
@app.callback(
    Output('grafik-kejahatan', 'figure'),
    Output('peta-kejahatan', 'figure'),
    Input('dropdown-tahun', 'value'),
    Input('dropdown-kejahatan', 'value')
)
def update_output(tahun, jenis):
    dff = df_melted[(df_melted['Tahun'] == tahun) & (df_melted['Jenis_Kejahatan'] == jenis)]

    fig_bar = px.bar(
        dff,
        x='Kabupaten',
        y='Jumlah',
        title=f'Jumlah Kejahatan: {jenis} Tahun {tahun}',
        labels={'Jumlah': 'Jumlah Kasus', 'Kabupaten': 'Kabupaten/Kota'}
    )

    fig_map = px.scatter_mapbox(
        dff,
        lat="Latitude",
        lon="Longitude",
        size="Jumlah",
        hover_name="Kabupaten",
        color="Jumlah",
        color_continuous_scale="OrRd",
        zoom=6,
        height=500
    )
    fig_map.update_layout(mapbox_style="open-street-map")
    fig_map.update_layout(margin={"r":0,"t":30,"l":0,"b":0}, title="Peta Kejahatan di Lampung")

    return fig_bar, fig_map

# Jalankan aplikasi
if __name__ == '__main__':
    app.run(debug=True)


<IPython.core.display.Javascript object>